Follows the how-to here-> "Of data preprocessing scheme for text-text (Mach Translation)"

In [2]:
import torch
import spacy
import torchtext
import pandas as pd

from sklearn.model_selection import train_test_split
from torchtext.data import Field, BucketIterator, TabularDataset

In [3]:
# Data is line delineated
en_text = open('../data/europarl-v7.fr-en.en', encoding='utf-8').read().split('\n')
fr_text = open('../data/europarl-v7.fr-en.fr', encoding='utf-8').read().split('\n')

In [4]:
# Load Tokenizers
en_tokenizer = spacy.load('en')
fr_tokenizer = spacy.load('fr')

def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]
def tokenize_fr(sentence):
    return [tok.text for tok in fr.tokenizer(sentence)]

In [5]:
EN_TEXT = Field(tokenize=tokenize_en)
FR_TEXT = Field(tokenize=tokenize_fr, init_token = "<sos>", eos_token = "<eos>")

In [6]:
# convert to dict fomrat for pandas
raw_data = {'English' : [line for line in europarl_en], 'French': [line for line in europarl_fr]}
df = pd.DataFrame(raw_data, columns=["English", "French"])

# remove very long sentences and sentences where translations are 
# not of roughly equal length
df['eng_len'] = df['English'].str.count(' ')
df['fr_len'] = df['French'].str.count(' ')
df = df.query('fr_len < 80 & eng_len < 80')
df = df.query('fr_len < eng_len * 1.5 & fr_len * 1.5 > eng_len')

In [7]:
# create train and validation set 
train, val = train_test_split(df, test_size=0.1)

train.to_csv("train.csv", index=False)
val.to_csv("val.csv", index=False)

In [9]:
%%time
# associate the text in the 'English' column with the EN_TEXT field, # and 'French' with FR_TEXT
data_fields = [('English', EN_TEXT), ('French', FR_TEXT)]
train,val = torchtext.data.TabularDataset.splits(path='./', train='train.csv', validation='val.csv', format='csv', fields=data_fields)

In [10]:
# Once finally done, we can index all the tokens:
FR_TEXT.build_vocab(train, val)
EN_TEXT.build_vocab(train, val)

In [13]:
#what it do?
print(EN_TEXT.vocab.stoi['the'])

2


In [14]:
# turn this shite into an iterator
train_iter = BucketIterator(train, batch_size=20, sort_key=lambda x: len(x.French), shuffle=True)


"""In each batch, the sentences have been transposed so they are descending 
vertically (important: we will need to transpose these again to work with the transformer). 
Each index represents a token (word), and each column represents a sentence. 
We have 10 columns, as 10 was the batch_size we specified."""

"""Additionally, 1 is used as a padding word here"""